In [179]:
import pandas as pd
import xlrd
import os 
import datetime
import folium 
import plotly.express as px
import plotly
import plotly.graph_objects as go
import argparse
import pyarrow
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from plotly.subplots import make_subplots
import plotly.figure_factory as ff
from functools import reduce

### Part 1. Load the eco driving and sensor tracing data

In [38]:
folder_path = "/Users/ruiwenzhang/Dropbox/PSV Rider Feedback/Data/Sensor Data/FinalData"
outputs_path = "/Users/ruiwenzhang/Desktop/Kenya/Weekly_report"
if not os.path.exists(outputs_path):
                os.mkdir(outputs_path)

eco_data_path = os.path.join(folder_path, "echodriving.gz.parquet")
sensor_tracing_path = os.path.join(folder_path, "sensortracing.gz.parquet")

In [39]:
eco_data_raw = pd.read_parquet(eco_data_path)
sensor_tracing_raw = pd.read_parquet(sensor_tracing_path)

### Part 2. Process the data

In [40]:
def data_process_eco(df):
    #df.Beginning = pd.to_datetime(df.Beginning.str.replace(".","-"), format = "%d-%m-%Y %H:%M:%S")
    df["begin_date"] = df.begin_datetime.dt.date
    df["begin_hour"] = df.begin_datetime.dt.hour
    #df.End = pd.to_datetime(df.End.str.replace(".","-"), format = "%d-%m-%Y %H:%M:%S")
    df["end_date"] = df.end_datetime.dt.date
    df["end_hour"] = df.end_datetime.dt.hour
    df["harsh_event"] = df.violation.str.split(" ", expand=True)[0]
    df["level"] = df.violation.str.split(" ", expand=True)[1]
   
    
    
    return df


def data_process_sensor(df):
    df.time_str = pd.to_datetime(df.time_str)
    df["date"] = df.time_str.dt.date
    df["hr"] = df.time_str.dt.hour
    df['time_delta'] = df.groupby(['reg_no'])['time_str'].diff()
    df['time_delta'] = df['time_delta'].apply(lambda x: x.total_seconds())
    
    return df

In [41]:
eco_data = data_process_eco(eco_data_raw)
sensor_tracing = data_process_sensor(sensor_tracing_raw)

In [42]:
eco_data.head()

,violation,begin_datetime,latitude_begin,longitude_begin,end_datetime,latitude_end,longitude_end,value_g,max_speed_kmhr,time_interval,distance_km,reg_no_id,reg_no,begin_date,begin_hour,end_date,end_hour,harsh_event,level
0,Brake: extreme,2021-07-01 05:14:45+03:00,-0.693320,37.340107,2021-07-01 05:14:45+03:00,-0.693368,37.340037,0.40,34.0,0:00:00,14.0,23672019,hiace kcf 954t,2021-07-01,5,2021-07-01,5,Brake:,extreme
1,Turn: medium,2021-07-01 08:00:47+03:00,-0.627922,37.022110,2021-07-01 08:00:48+03:00,-0.627978,37.022003,0.34,35.0,0:00:01,49.0,23457115,hiace kcw 543m,2021-07-01,8,2021-07-01,8,Turn:,medium
2,Brake: medium,2021-07-01 08:18:56+03:00,-0.641155,37.082898,2021-07-01 08:18:57+03:00,-0.641157,37.082925,0.32,8.0,0:00:01,57.0,23457115,hiace kcw 543m,2021-07-01,8,2021-07-01,8,Brake:,medium
3,Brake: extreme,2021-07-01 08:33:04+03:00,-0.674540,37.116143,2021-07-01 08:33:04+03:00,-0.674595,37.116137,0.40,18.0,0:00:00,63.0,23457115,hiace kcw 543m,2021-07-01,8,2021-07-01,8,Brake:,extreme
4,Turn: extreme,2021-07-01 08:40:59+03:00,-0.710713,37.153908,2021-07-01 08:40:59+03:00,-0.710785,37.153978,0.41,45.0,0:00:00,70.0,23457115,hiace kcw 543m,2021-07-01,8,2021-07-01,8,Turn:,extreme


In [43]:
sensor_tracing.head()

,speed_kmhr,latitude,longitude,location,time_str,reg_no_id,reg_no,date,hr,time_delta
0,0.0,-0.591727,37.022522,"Muranga Kiriaini Roads, 4.63 km from Mukurweini",2021-07-19 00:00:02,23256164,hiace kcq 827w,2021-07-19,0,NaN
1,0.0,-0.591727,37.022522,"Muranga Kiriaini Roads, 4.63 km from Mukurweini",2021-07-19 00:00:12,23256164,hiace kcq 827w,2021-07-19,0,10.0
2,0.0,-0.591727,37.022522,"Muranga Kiriaini Roads, 4.63 km from Mukurweini",2021-07-19 00:00:22,23256164,hiace kcq 827w,2021-07-19,0,10.0
3,0.0,-0.591727,37.022522,"Muranga Kiriaini Roads, 4.63 km from Mukurweini",2021-07-19 00:00:32,23256164,hiace kcq 827w,2021-07-19,0,10.0
4,0.0,-0.591727,37.022522,"Muranga Kiriaini Roads, 4.63 km from Mukurweini",2021-07-19 00:00:42,23256164,hiace kcq 827w,2021-07-19,0,10.0


In [60]:
eco_data.reg_no.nunique()

20

In [61]:
sensor_tracing.reg_no.nunique()

20

In [63]:
for i in eco_data.violation.unique():
    print(i)

Brake: extreme
Turn: medium
Brake: medium
Turn: extreme
Acceleration: extreme
Acceleration: medium
Braking
Acceleration
Turn
Harsh driving
Speeding: mild
Speeding: medium
Speeding: extreme


In [12]:
sample = eco_data[eco_data.reg_no == 'hiace kcf 954t']
sample

,violation,begin_datetime,latitude_begin,longitude_begin,end_datetime,latitude_end,longitude_end,value_g,max_speed_kmhr,time_interval,distance_km,reg_no_id,reg_no,begin_date,begin_hour,end_date,end_hour,harsh_event,level
0,Brake: extreme,2021-07-01 05:14:45+03:00,-0.693320,37.340107,2021-07-01 05:14:45+03:00,-0.693368,37.340037,0.40,34.0,0:00:00,14.00,23672019,hiace kcf 954t,2021-07-01,5,2021-07-01,5,Brake:,extreme
370,Acceleration: extreme,2021-07-03 04:59:19+03:00,-0.507330,37.459835,2021-07-03 04:59:19+03:00,-0.507272,37.459838,0.61,36.0,0:00:00,2.59,23672019,hiace kcf 954t,2021-07-03,4,2021-07-03,4,Acceleration:,extreme
2349,Brake: extreme,2021-07-08 13:30:45+03:00,-0.958522,37.110348,2021-07-08 13:30:45+03:00,-0.958443,37.110423,0.40,41.0,0:00:00,176.00,23672019,hiace kcf 954t,2021-07-08,13,2021-07-08,13,Brake:,extreme
2350,Brake: extreme,2021-07-08 16:39:28+03:00,-0.475967,37.464923,2021-07-08 16:39:28+03:00,-0.475913,37.464962,0.40,24.0,0:00:00,259.00,23672019,hiace kcf 954t,2021-07-08,16,2021-07-08,16,Brake:,extreme
2351,Brake: extreme,2021-07-08 17:16:39+03:00,-0.449422,37.557343,2021-07-08 17:16:39+03:00,-0.449338,37.557318,0.40,31.0,0:00:00,273.00,23672019,hiace kcf 954t,2021-07-08,17,2021-07-08,17,Brake:,extreme
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25677,Brake: extreme,2021-08-11 02:23:22+03:00,-0.336292,37.645162,2021-08-11 02:23:22+03:00,-0.336227,37.645270,0.53,46.0,0:00:00,258.00,23672019,hiace kcf 954t,2021-08-11,2,2021-08-11,2,Brake:,extreme
25678,Acceleration: medium,2021-08-11 02:24:02+03:00,-0.334235,37.644938,2021-08-11 02:24:02+03:00,-0.334203,37.644897,0.36,23.0,0:00:00,259.00,23672019,hiace kcf 954t,2021-08-11,2,2021-08-11,2,Acceleration:,medium
25679,Brake: extreme,2021-08-11 02:24:30+03:00,-0.333257,37.643287,2021-08-11 02:24:30+03:00,-0.333252,37.643248,0.40,11.0,0:00:00,259.00,23672019,hiace kcf 954t,2021-08-11,2,2021-08-11,2,Brake:,extreme
25680,Acceleration: extreme,2021-08-11 02:28:15+03:00,-0.333770,37.644373,2021-08-11 02:28:15+03:00,-0.333753,37.644342,0.45,21.0,0:00:00,260.00,23672019,hiace kcf 954t,2021-08-11,2,2021-08-11,2,Acceleration:,extreme


### Part 3. Plots
* Eco driving: Boxplot of max speed and acceleration across violation types 

* Sensor tracing: Boxplot of speed

In [53]:
def boxplot_maxsp_acc(df, vh_list):
    def plot(df,vh,export = True, show = False):
        fig = make_subplots(rows = 2, cols = 1)
        
        fig.add_trace(go.Box(
        x = df[df.reg_no == vh].violation.values,
        y = df[df.reg_no == vh].value_g.values,
        name='acceleration',
        marker_color='#3D9970',boxpoints='suspectedoutliers'
        ), row=1, col=1)
        
        fig.add_trace(go.Box(
        x = df[df.reg_no == vh].violation.values,
        y = df[df.reg_no == vh].max_speed_kmhr.values,
        name='max speed',
        marker_color='#FF4136',boxpoints='suspectedoutliers'
        ), row=2, col=1)
        
        fig.update_layout(height=800, width=1000, 
        title_text = vh + ": Boxplot of Max Speed & Acceleration Value across Violation Types")
        
        if export:
            folder_path = outputs_path + '/' + vh
            if not os.path.exists(folder_path):
                os.mkdir(folder_path)
            file_name = folder_path + '/' + 'boxplot_maxsp_acc_' + vh + '.html'
            print('Saving: ' + file_name)
            plotly.offline.plot(fig, filename = file_name, auto_open=False)
        if show:
            fig.show()
    
    for vh in vh_list:
        box_maxsp_acc = plot(df,vh)
        

In [57]:
def boxplot_speed(df,vh_list):
     def plot(df,vh,export = True, show = False):
        speed = df[df.reg_no == vh]['speed_kmhr']
        fig = go.Figure()
    
        fig.add_trace(go.Box(y = speed, name = 'speed',boxpoints='suspectedoutliers'))
        fig.update_layout(title_text="Speed Boxplot of " + vh)
        
        
        if export:
            folder_path = outputs_path + '/' + vh
            if not os.path.exists(folder_path):
                os.mkdir(folder_path)
            file_name = folder_path + '/' + 'boxplot_speed_' + vh + '.html'
            print('Saving: ' + file_name)
            plotly.offline.plot(fig, filename = file_name, auto_open=False)
        if show:
            fig.show()
            
     for vh in vh_list:
        speed_box = plot(df,vh)

In [54]:
vh_list = eco_data.reg_no.unique()
boxplot_maxsp_acc(eco_data, vh_list)

Saving: /Users/ruiwenzhang/Desktop/Kenya/Weekly_report/hiace kcf 954t/boxplot_maxsp_acc_hiace kcf 954t.html
Saving: /Users/ruiwenzhang/Desktop/Kenya/Weekly_report/hiace kcw 543m/boxplot_maxsp_acc_hiace kcw 543m.html
Saving: /Users/ruiwenzhang/Desktop/Kenya/Weekly_report/noah kcr 318t/boxplot_maxsp_acc_noah kcr 318t.html
Saving: /Users/ruiwenzhang/Desktop/Kenya/Weekly_report/hiace kcz 255g/boxplot_maxsp_acc_hiace kcz 255g.html
Saving: /Users/ruiwenzhang/Desktop/Kenya/Weekly_report/hiace kdc 389h/boxplot_maxsp_acc_hiace kdc 389h.html
Saving: /Users/ruiwenzhang/Desktop/Kenya/Weekly_report/hiace kcl 589a/boxplot_maxsp_acc_hiace kcl 589a.html
Saving: /Users/ruiwenzhang/Desktop/Kenya/Weekly_report/noah kcz 750e/boxplot_maxsp_acc_noah kcz 750e.html
Saving: /Users/ruiwenzhang/Desktop/Kenya/Weekly_report/hiace kcq 827w/boxplot_maxsp_acc_hiace kcq 827w.html
Saving: /Users/ruiwenzhang/Desktop/Kenya/Weekly_report/hiace kcq 975j/boxplot_maxsp_acc_hiace kcq 975j.html
Saving: /Users/ruiwenzhang/Deskt

In [58]:
boxplot_speed(sensor_tracing, vh_list)

Saving: /Users/ruiwenzhang/Desktop/Kenya/Weekly_report/hiace kcf 954t/boxplot_speed_hiace kcf 954t.html
Saving: /Users/ruiwenzhang/Desktop/Kenya/Weekly_report/hiace kcw 543m/boxplot_speed_hiace kcw 543m.html
Saving: /Users/ruiwenzhang/Desktop/Kenya/Weekly_report/noah kcr 318t/boxplot_speed_noah kcr 318t.html
Saving: /Users/ruiwenzhang/Desktop/Kenya/Weekly_report/hiace kcz 255g/boxplot_speed_hiace kcz 255g.html
Saving: /Users/ruiwenzhang/Desktop/Kenya/Weekly_report/hiace kdc 389h/boxplot_speed_hiace kdc 389h.html
Saving: /Users/ruiwenzhang/Desktop/Kenya/Weekly_report/hiace kcl 589a/boxplot_speed_hiace kcl 589a.html
Saving: /Users/ruiwenzhang/Desktop/Kenya/Weekly_report/noah kcz 750e/boxplot_speed_noah kcz 750e.html
Saving: /Users/ruiwenzhang/Desktop/Kenya/Weekly_report/hiace kcq 827w/boxplot_speed_hiace kcq 827w.html
Saving: /Users/ruiwenzhang/Desktop/Kenya/Weekly_report/hiace kcq 975j/boxplot_speed_hiace kcq 975j.html
Saving: /Users/ruiwenzhang/Desktop/Kenya/Weekly_report/hiace kcx 726

In [30]:
fig = make_subplots(rows = 2, cols = 1)

fig.add_trace(go.Box(
    x=df[df.reg_no == vh].violation.values,
    y=sample.value_g.values,
    name='acceleration',
    marker_color='#3D9970',boxpoints='suspectedoutliers'
), row=1, col=1)

fig.add_trace(go.Box(
    x=sample.violation.values,
    y=sample.max_speed_kmhr.values,
    name='max speed',
    marker_color='#FF4136',boxpoints='suspectedoutliers'
), row=2, col=1)


fig.update_layout(height=800, width=1000, 
                  title_text="Boxplot of Max Speed & Acceleration Value across Violation Types)
fig.show()
#fig.update_layout(
#    yaxis_title='normalized moisture',
#    boxmode='group' # group together boxes of the different traces for each value of x
#)

### Part 4. Create a report

* drop the rows where the violation is speeding or harsh driving

In [193]:
drop_rows = (eco_data.violation.str.startswith('Speeding') | eco_data.violation.str.startswith('Harsh'))
eco_data_reprt = eco_data[~drop_rows]
eco_data_reprt

,violation,begin_datetime,latitude_begin,longitude_begin,end_datetime,latitude_end,longitude_end,value_g,max_speed_kmhr,time_interval,distance_km,reg_no_id,reg_no,begin_date,begin_hour,end_date,end_hour,harsh_event,level
0,Brake: extreme,2021-07-01 05:14:45+03:00,-0.693320,37.340107,2021-07-01 05:14:45+03:00,-0.693368,37.340037,0.40,34.0,0:00:00,14.0,23672019,hiace kcf 954t,2021-07-01,5,2021-07-01,5,Brake:,extreme
1,Turn: medium,2021-07-01 08:00:47+03:00,-0.627922,37.022110,2021-07-01 08:00:48+03:00,-0.627978,37.022003,0.34,35.0,0:00:01,49.0,23457115,hiace kcw 543m,2021-07-01,8,2021-07-01,8,Turn:,medium
2,Brake: medium,2021-07-01 08:18:56+03:00,-0.641155,37.082898,2021-07-01 08:18:57+03:00,-0.641157,37.082925,0.32,8.0,0:00:01,57.0,23457115,hiace kcw 543m,2021-07-01,8,2021-07-01,8,Brake:,medium
3,Brake: extreme,2021-07-01 08:33:04+03:00,-0.674540,37.116143,2021-07-01 08:33:04+03:00,-0.674595,37.116137,0.40,18.0,0:00:00,63.0,23457115,hiace kcw 543m,2021-07-01,8,2021-07-01,8,Brake:,extreme
4,Turn: extreme,2021-07-01 08:40:59+03:00,-0.710713,37.153908,2021-07-01 08:40:59+03:00,-0.710785,37.153978,0.41,45.0,0:00:00,70.0,23457115,hiace kcw 543m,2021-07-01,8,2021-07-01,8,Turn:,extreme
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26486,Acceleration: extreme,2021-08-10 14:25:29+03:00,-0.517383,36.940903,2021-08-10 14:25:29+03:00,-0.517332,36.940887,0.40,21.0,0:00:00,34.0,23146908,hiace kcl 589a,2021-08-10,14,2021-08-10,14,Acceleration:,extreme
26487,Acceleration: medium,2021-08-10 14:32:00+03:00,-0.501545,36.938907,2021-08-10 14:32:00+03:00,-0.501545,36.938920,0.32,14.0,0:00:00,37.0,23146908,hiace kcl 589a,2021-08-10,14,2021-08-10,14,Acceleration:,medium
26488,Brake: medium,2021-08-10 14:33:49+03:00,-0.494955,36.940573,2021-08-10 14:33:49+03:00,-0.494922,36.940643,0.32,26.0,0:00:00,38.0,23146908,hiace kcl 589a,2021-08-10,14,2021-08-10,14,Brake:,medium
26489,Brake: extreme,2021-08-10 17:40:24+03:00,-0.452647,36.958742,2021-08-10 17:40:24+03:00,-0.452693,36.958710,0.40,21.0,0:00:00,59.0,23146908,hiace kcl 589a,2021-08-10,17,2021-08-10,17,Brake:,extreme


#### Total eco driving obs

In [194]:
eco_count = eco_data_reprt.groupby(["reg_no"]).apply(lambda x: pd.Series({
    "eco_count": x["reg_no"].count()
})).reset_index()
eco_count

,reg_no,eco_count
0,hiace kcf 954t,689
1,hiace kcj 537z,224
2,hiace kcl 589a,172
3,hiace kcm 320j,1122
4,hiace kcn 863s,206
5,hiace kcq 580g,204
6,hiace kcq 827w,356
7,hiace kcq 975j,9422
8,hiace kcr 120f,109
9,hiace kct 045g,4


#### Total sensor tracing obs

In [195]:
sensor_count = sensor_tracing.groupby(["reg_no"]).apply(lambda x: pd.Series({
    "sensor_count": x["reg_no_id"].count()
})).reset_index()
sensor_count

,reg_no,sensor_count
0,hiace kcf 954t,218712
1,hiace kcj 537z,126666
2,hiace kcl 589a,187502
3,hiace kcm 320j,22318
4,hiace kcn 863s,156683
5,hiace kcq 580g,141486
6,hiace kcq 827w,211676
7,hiace kcq 975j,213788
8,hiace kcr 120f,146287
9,hiace kct 045g,7741


In [196]:
eco_data_reprt.harsh_event.unique()

array(['Brake:', 'Turn:', 'Acceleration:', 'Braking', 'Acceleration',
       'Turn'], dtype=object)

#### categorize braking to brake: harsh event, turn to turn: harsh event, acceleration to acceleration: harsh event
#### Note:  brake: harsh event include braking and brake with extreme and medium level, same for the turn: and acc:

In [183]:
eco_data_reprt['harsh_event'] = eco_data_reprt['harsh_event'].\
                                replace({'Braking':'Brake:', 'Turn':'Turn:', 'Acceleration': 'Acceleration:'})

#### the total obs for each violation type each vehicle

In [184]:
vio_type_cnt = eco_data_reprt.groupby(['reg_no','harsh_event']).apply(lambda x: pd.Series({
    "count": x["reg_no"].count()
})).reset_index()
vio_type_cnt

,reg_no,harsh_event,count
0,hiace kcf 954t,Acceleration:,209
1,hiace kcf 954t,Brake:,437
2,hiace kcf 954t,Turn:,43
3,hiace kcj 537z,Acceleration:,82
4,hiace kcj 537z,Brake:,126
5,hiace kcj 537z,Turn:,16
6,hiace kcl 589a,Acceleration:,46
7,hiace kcl 589a,Brake:,108
8,hiace kcl 589a,Turn:,18
9,hiace kcm 320j,Acceleration:,274


* Reshape the df

In [185]:
vio_type_cnt = vio_type_cnt.pivot(index = 'reg_no', columns = 'harsh_event', values = 'count').\
            rename_axis(None,axis=1).reset_index()
vio_type_cnt

,reg_no,Acceleration:,Brake:,Turn:
0,hiace kcf 954t,209.0,437.0,43.0
1,hiace kcj 537z,82.0,126.0,16.0
2,hiace kcl 589a,46.0,108.0,18.0
3,hiace kcm 320j,274.0,649.0,199.0
4,hiace kcn 863s,71.0,116.0,19.0
5,hiace kcq 580g,89.0,76.0,39.0
6,hiace kcq 827w,111.0,147.0,98.0
7,hiace kcq 975j,2429.0,5147.0,1846.0
8,hiace kcr 120f,42.0,57.0,10.0
9,hiace kct 045g,1.0,2.0,1.0


#### the total obs for each violation type with level for each vehicle 

In [186]:
vio_type_level = eco_data_reprt.groupby(['reg_no','violation']).apply(lambda x: pd.Series({
    "count": x["reg_no"].count()})).reset_index()
#vio_type_level = vio_type_level.pivot(index = 'reg_no', columns = 'violation', values = 'count').\
#            rename_axis(None,axis=1).reset_index()
vio_type_level

,reg_no,violation,count
0,hiace kcf 954t,Brake: extreme,345.0
1,hiace kcf 954t,Turn: medium,22.0
2,hiace kcf 954t,Brake: medium,92.0
3,hiace kcf 954t,Turn: extreme,21.0
4,hiace kcf 954t,Acceleration: extreme,180.0
...,...,...,...
255,sprinter kch 471d,Turn,NaN
256,sprinter kch 471d,Harsh driving,NaN
257,sprinter kch 471d,Speeding: mild,NaN
258,sprinter kch 471d,Speeding: medium,NaN


In [187]:
vio_type_level = vio_type_level.pivot(index = 'reg_no', columns = 'violation', values = 'count').\
rename_axis(None,axis=1)
vio_type_level.columns = vio_type_level.columns.astype(str)
vio_type_level = vio_type_level.reset_index().drop(["Harsh driving","Speeding: mild"
                                                    ,"Speeding: medium","Speeding: extreme"], axis = 1)

#### merge multiple dfs and create the report

In [189]:
dfs = [eco_count,sensor_count,vio_type_cnt,vio_type_level]

In [190]:
report = reduce(lambda x,y: pd.merge(x,y, on='reg_no', how='outer'), dfs)
report

,reg_no,eco_count,sensor_count,Acceleration:,Brake:,Turn:,Brake: extreme,Turn: medium,Brake: medium,Turn: extreme,Acceleration: extreme,Acceleration: medium,Braking,Acceleration,Turn
0,hiace kcf 954t,689,218712,209.0,437.0,43.0,345.0,22.0,92.0,21.0,180.0,29.0,NaN,NaN,NaN
1,hiace kcj 537z,224,126666,82.0,126.0,16.0,97.0,14.0,29.0,2.0,66.0,16.0,NaN,NaN,NaN
2,hiace kcl 589a,172,187502,46.0,108.0,18.0,78.0,10.0,30.0,8.0,32.0,14.0,NaN,NaN,NaN
3,hiace kcm 320j,1122,22318,274.0,649.0,199.0,NaN,NaN,NaN,NaN,NaN,NaN,649.0,274.0,199.0
4,hiace kcn 863s,206,156683,71.0,116.0,19.0,95.0,10.0,21.0,9.0,54.0,17.0,NaN,NaN,NaN
5,hiace kcq 580g,204,141486,89.0,76.0,39.0,56.0,24.0,20.0,15.0,59.0,30.0,NaN,NaN,NaN
6,hiace kcq 827w,356,211676,111.0,147.0,98.0,121.0,68.0,26.0,30.0,81.0,30.0,NaN,NaN,NaN
7,hiace kcq 975j,9422,213788,2429.0,5147.0,1846.0,163.0,34.0,48.0,19.0,NaN,44.0,4936.0,2385.0,1793.0
8,hiace kcr 120f,109,146287,42.0,57.0,10.0,49.0,6.0,8.0,4.0,31.0,11.0,NaN,NaN,NaN
9,hiace kct 045g,4,7741,1.0,2.0,1.0,2.0,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN


In [202]:
fig = go.Figure(data=[go.Table(
    header=dict(values=list(report.columns),
                fill_color='paleturquoise',
                align='left'),
    cells=dict(values=[report['reg_no'], report['eco_count'], report['sensor_count'], report['Acceleration:'],
                      report['Brake:'], report['Turn:'], report['Brake: extreme'],report['Turn: medium'],
                       report['Brake: medium'],report['Turn: extreme'],report['Acceleration: extreme'],
                       report['Acceleration: medium'], report['Braking'], report['Acceleration'], report['Turn']],
               fill_color='lavender',
               align='left'))
])

#folder_path = outputs_path + '/' + vh
#if not os.path.exists(folder_path):
#    os.mkdir(folder_path)
file_name = outputs_path + '/' + 'report.html'
print('Saving: ' + file_name)
plotly.offline.plot(fig, filename = file_name, auto_open=False)
fig.show()

Saving: /Users/ruiwenzhang/Desktop/Kenya/Weekly_report/report.html
